In [1]:
import pandas as pd
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.compose import make_column_transformer 
from sklearn.compose import make_column_selector
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import KNNImputer, SimpleImputer
import warnings
warnings.filterwarnings('ignore')

In [13]:
satellite = pd.read_csv("Satellite.csv", sep=";")
satellite

,x.1,x.2,x.3,x.4,x.5,x.6,x.7,x.8,x.9,x.10,...,x.28,x.29,x.30,x.31,x.32,x.33,x.34,x.35,x.36,classes
0,92,115,120,94,84,102,106,79,84,102,...,104,88,121,128,100,84,107,113,87,grey soil
1,84,102,106,79,84,102,102,83,80,102,...,100,84,107,113,87,84,99,104,79,grey soil
2,84,102,102,83,80,102,102,79,84,94,...,87,84,99,104,79,84,99,104,79,grey soil
3,80,102,102,79,84,94,102,79,80,94,...,79,84,99,104,79,84,103,104,79,grey soil
4,84,94,102,79,80,94,98,76,80,102,...,79,84,103,104,79,79,107,109,87,grey soil
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6430,60,83,96,85,64,87,100,88,64,83,...,92,66,87,108,89,63,83,104,85,red soil
6431,64,79,100,85,56,71,96,85,56,68,...,85,66,83,100,85,63,83,100,81,red soil
6432,56,68,91,81,56,64,91,81,53,64,...,81,59,87,96,81,63,83,92,74,vegetation stubble
6433,56,68,87,74,60,71,91,81,60,64,...,74,59,83,92,74,59,83,92,70,vegetation stubble


In [43]:
x = satellite.drop('classes', axis = 'columns')
y = satellite['classes']

In [45]:
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                   random_state = 24,
                                                   test_size = 0.3,
                                                   stratify = y)

In [47]:
lda = LinearDiscriminantAnalysis().set_output(transform = 'pandas')
lda.fit(x_train, y_train)

LinearDiscriminantAnalysis()

In [53]:
y_pred = lda.predict(x_test)

In [55]:
print(accuracy_score(y_test,y_pred))

0.8296219575349559


In [39]:
y_pred_lr = lr.predict(lda.transform(x_test))
accuracy_score(y_test, y_pred_lr)

0.8575867426204039

In [33]:
# QDA
qda = QuadraticDiscriminantAnalysis()
qda.fit(x_train, y_train)
y_pred = qda.predict(x_test)
accuracy_score(y_test, y_pred)

0.8508544795442776

In [59]:
# Using knn
# For KNN

knn = KNeighborsClassifier(n_neighbors=3)

lda = LinearDiscriminantAnalysis()
pipe = Pipeline([('LDA', lda),('KNN',knn)])

pipe.fit(x_train, y_train)
y_pred= pipe.predict(x_test)
print(accuracy_score(y_test,y_pred))

0.8663904712584153


In [65]:
params = {'KNN__n_neighbors': np.arange(1,9)}
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=24)
gcv = GridSearchCV(pipe, param_grid=params, cv=kfold)
gcv.fit(x,y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('LDA', LinearDiscriminantAnalysis()),
                                       ('KNN',
                                        KNeighborsClassifier(n_neighbors=3))]),
             param_grid={'KNN__n_neighbors': array([1, 2, 3, 4, 5, 6, 7, 8])})

In [67]:
gcv.best_score_

0.8822066822066823

In [69]:
gcv.best_params_                                                                                          

{'KNN__n_neighbors': 7}